In [30]:
from IPython.display import display
import pandas as pd

pd.set_option('display.max_rows', None)

with open("../smart_app_data.pkl", "rb") as file:
    df = pd.read_pickle(file)

In [31]:
class kpi_dataframe_filter:
    def filter_dataframe_by_machine(df, machine_id):
        if machine_id != 'all_machines':
            return df[df['asset_id'] == machine_id]
        return dfw

    def filter_dataframe_by_kpi(df, kpi):
        return df[df['kpi'] == kpi]

    def filter_dataframe_by_time(df, start_time, end_time):
        return df[(df['time'] >= start_time) & (df['time'] <= end_time)]

In [32]:
class kpi_dataframe_utils:
    def sum_kpi(kpi, df, machine_id, start_time, end_time):
        fd = df     # fd = filtered dataframe
        fd = kpi_dataframe_filter.filter_dataframe_by_machine(fd, machine_id)
        fd = kpi_dataframe_filter.filter_dataframe_by_kpi(fd, kpi)
        fd = kpi_dataframe_filter.filter_dataframe_by_time(fd, start_time, end_time)
        '''
        if machine_id != 'all_machines': 
            print(f"KPI calculated as sum on machine {machine_id} ({fd.iloc[0, 2]}) by summing {kpi} from {start_time} to {end_time}. Result is {fd['sum'].sum()}.")
        else: 
            print(f"KPI calculated as sum on machine {machine_id} by summing {kpi} from {start_time} to {end_time}. Result is {fd['sum'].sum()}.")
        '''
            
        return fd['sum'].sum()


    def avg_kpi(kpi, df, machine_id, start_time, end_time):
        fd = df     # fd = filtered dataframe
        fd = kpi_dataframe_filter.filter_dataframe_by_machine(fd, machine_id)
        fd = kpi_dataframe_filter.filter_dataframe_by_kpi(fd, kpi)
        fd = kpi_dataframe_filter.filter_dataframe_by_time(fd, start_time, end_time)
        '''
        print(f"KPI calculated as average on machine {machine_id} ({fd.iloc[0, 2]}) by averaging {kpi}s from {start_time} to {end_time}. Result is {fd['avg'].sum()/fd.shape[0]}.")
        '''
        return fd['avg'].sum()/fd.shape[0]

    def max_kpi(kpi, df, machine_id, start_time, end_time):
        fd = df #fd  = filtered dataframe
        fd = kpi_dataframe_filter.filter_dataframe_by_machine(fd, machine_id)
        fd = kpi_dataframe_filter.filter_dataframe_by_kpi(fd, kpi)
        fd = kpi_dataframe_filter.filter_dataframe_by_time(fd, start_time, end_time)
        '''
        print(f"KPI calculated on machine {machine_id} ({fd.iloc[0, 2]}) as maximum {kpi}s from {start_time} to {end_time}. Result is {fd['sum'].max()}.")
        '''
        return fd['sum'].max()


    def min_kpi(kpi, df, machine_id, start_time, end_time):
        fd = df #fd  = filtered dataframe
        fd = kpi_dataframe_filter.filter_dataframe_by_machine(fd, machine_id)
        fd = kpi_dataframe_filter.filter_dataframe_by_kpi(fd, kpi)
        fd = kpi_dataframe_filter.filter_dataframe_by_time(fd, start_time, end_time)
        '''
        print(f"KPI calculated on machine {machine_id} ({fd.iloc[0, 2]}) as minimum {kpi}s from {start_time} to {end_time}. Result is {fd['sum'].min()}.")
        '''
        fd['sum'].min()

In [33]:
class kpi_engine:
    def energy_cost_savings(df, machine_id, time):
        fd = df
        fd = kpi_dataframe_filter.filter_dataframe_by_machine(fd, machine_id)
        fd = kpi_dataframe_filter.filter_dataframe_by_kpi(fd, 'cost')

        index = fd['time'].str.contains(time, na=False).idxmax()
        ## if index is the first of the dataframe there is no previous time measurement, return error
        return fd.at[fd.index[fd.index.get_loc(index) - 1], 'sum'] - fd.at[index, 'sum']

    def energy_cost_working_time(df, start_time, end_time):
        fd = df
        total_energy_cost = kpi_dataframe_utils.sum_kpi(df=fd, kpi='cost_working', machine_id='all_machines', start_time=start_time, end_time=end_time)
        total_working_time = kpi_dataframe_filter.filter_dataframe_by_time(df=fd, start_time=start_time, end_time=end_time)['time'].nunique() * 24
        return  total_energy_cost / total_working_time

    def energy_cost_idle_time(df, start_time, end_time):
        fd = df
        total_energy_cost = kpi_dataframe_utils.sum_kpi(df=fd, kpi='cost_idle', machine_id='all_machines', start_time=start_time, end_time=end_time)
        total_working_time = kpi_dataframe_utils.sum_kpi(df=fd, kpi='working_time', machine_id='all_machines', start_time=start_time, end_time=end_time)
        return  total_energy_cost / total_working_time

    def energy_cost_per_unit(df, machine_id, start_time, end_time):
        fd = df
        total_working_cost = kpi_dataframe_utils.sum_kpi(df=fd, kpi='cost_working', machine_id=machine_id, start_time=start_time, end_time=end_time)
        total_idle_cost = kpi_dataframe_utils.sum_kpi(df=fd, kpi='cost_idle', machine_id=machine_id, start_time=start_time, end_time=end_time)
        return total_working_cost + total_idle_cost

    def power_consumption_efficiency(df, machine_id, start_time, end_time):
        fd = df
        total_working_time = kpi_dataframe_utils.sum_kpi(kpi='working_time', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        total_power_consumption = kpi_dataframe_utils.sum_kpi(df=fd, kpi='consumption', machine_id=machine_id, start_time=start_time, end_time=end_time)
        return total_working_time / total_power_consumption
        
    def power_consumption_trend(df, machine_id, start_previous_period, end_previous_period, start_current_period, end_current_period):
        fd = df
        if(not(start_previous_period <= end_previous_period < start_current_period <= end_current_period)):
            print("bad chronological order")
        current_total_power_consumption = kpi_dataframe_utils.sum_kpi(df=fd, kpi='consumption', machine_id=machine_id, start_time=start_current_period, end_time=end_current_period)
        previous_total_power_consumption = kpi_dataframe_utils.sum_kpi(df=fd, kpi='consumption', machine_id=machine_id, start_time=start_previous_period, end_time=end_previous_period)
        return (current_total_power_consumption - previous_total_power_consumption) / previous_total_power_consumption

    def machine_utiization_rate(df, machine_id, start_time, end_time):
        fd = df
        total_working_time = kpi_dataframe_utils.sum_kpi(kpi='working_time', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        total_idle_time = kpi_dataframe_utils.sum_kpi(kpi='idle_time', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        total_offline_time = kpi_dataframe_utils.sum_kpi(kpi='offline_time', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        return total_working_time / (total_working_time + total_idle_time + total_offline_time)

    def machine_usage_trend(df, machine_id, start_previous_period, end_previous_period, start_current_period, end_current_period):
        fd = df
        if(not(start_previous_period <= end_previous_period < start_current_period <= end_current_period)):
            print("bad chronological order")
        current_average_working_time = kpi_dataframe_utils.avg_kpi(df=fd, kpi='working_time', machine_id=machine_id, start_time=start_current_period, end_time=end_current_period)
        previous_average_working_time = kpi_dataframe_utils.avg_kpi(df=fd, kpi='working_time', machine_id=machine_id, start_time=start_previous_period, end_time=end_previous_period)
        return (current_average_working_time - previous_average_working_time) / previous_average_working_time

    '''
    def cost_per_unit():
        return -1

    def material_cost_per_unit():
        return -1

    def material_cost_per_unit():
        return -1
    '''

    def availability(df, machine_id, start_time, end_time):
        fd = df
        uptime = kpi_dataframe_utils.sum_kpi(kpi='working_time', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        downtime = kpi_dataframe_utils.sum_kpi(kpi='idle_time', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        return uptime / (uptime + downtime)

    def performance(df, machine_id, start_time, end_time):
        fd = df
        total_output = kpi_dataframe_utils.sum_kpi(kpi='good_cycles', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        total_productive_time = kpi_dataframe_utils.sum_kpi(kpi='working_time', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        return total_output / total_productive_time

    def throughput(df, machine_id, start_time, end_time):
        fd = df
        items_produced = kpi_dataframe_utils.sum_kpi(kpi='good_cycles', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        time_employed = kpi_dataframe_utils.sum_kpi(kpi='working_time', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time) + kpi_dataframe_utils.sum_kpi(kpi='idle_time', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        return items_produced / time_employed

    def quality(df, machine_id, start_time, end_time):
        fd = df
        good_work = kpi_dataframe_utils.sum_kpi(kpi='good_cycles', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        bad_work = kpi_dataframe_utils.sum_kpi(kpi='bad_cycles', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        total_work = good_work + bad_work
        return good_work / total_work

    def yield_fft(df, machine_id, start_time, end_time):
        fd = df
        defective_output = kpi_dataframe_utils.sum_kpi(kpi='bad_cycles', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time)
        total_output = kpi_dataframe_utils.sum_kpi(kpi='good_cycles', df=fd, machine_id=machine_id, start_time=start_time, end_time=end_time) + defective_output
        return (total_output - defective_output) / total_output

    '''
    def maintenance_cost():
        return -1

    def mean_time_between_failures():
        return -1

    def mean_time_between_maintenance():
        return -1

    def mean_time_to_repair():
        return -1
    '''

In [38]:
'''
kpi_engine.sum_kpi(kpi='cost_working', df=df, machine_id='all_machines', start_time='' , end_time= '9')

kpi_engine.avg_kpi(kpi='power', df=df, machine_id='ast-xpimckaf3dlf', start_time='2024-04-07T00:00:00Z', end_time='2024-07-09T00:00:00Z')

kpi_engine.max_kpi(kpi='bad_cycles',df=df, machine_id='ast-xpimckaf3dlf',start_time='2024-04-07T00:00:00Z', end_time='2024-07-09T00:00:00Z')

kpi_engine.min_kpi(kpi='good_cycles',df=df, machine_id='ast-xpimckaf3dlf',start_time='2024-04-07T00:00:00Z', end_time='2024-07-09T00:00:00Z')

print(kpi_engine.energy_cost_per_unit(df=df, machine_id='ast-06kbod797nnp', start_time='2024-08-28T00:00:00Z', end_time='2024-09-27T00:00:00Z'))
'''
print(kpi_engine.machine_usage_trend(df=df, machine_id='ast-06kbod797nnp', start_previous_period='2024-08-29T00:00:00Z', end_previous_period='2024-08-29T00:00:00Z', start_current_period='2024-08-30T00:00:00Z', end_current_period='2024-08-30T00:00:00Z'))

0.10561096601167418
